In [0]:
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import json 
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
ls

 Alldone_1hot_v1.csv           Proposal.gdoc
'Data Dictionary .gdoc'        restaurantbdata.csv
'facebook data'/               restaurantreviews.csv
 FBPreprocessing.ipynb        'Restaurant Success Prediction.gslides'
 Final_merged_FB_Yelp_v1.csv   restaurant_toronto.csv
 FinalPreprocessing.ipynb      ReviewsPreprocessing.ipynb
'old data'/                    shehryarSVM.pkl
 processsedTorontoFB.csv       Yelp_Filtered_SA_Dataset.csv
'Project Progress.gdoc'        yelp_preprocessing.ipynb


In [4]:
cd drive/Shared\ drives/

/content/drive/Shared drives


In [0]:
%ls

'Data Science Principles Project'/   Multicore/


In [5]:
cd 'Data Science Principles Project'

/content/drive/Shared drives/Data Science Principles Project


In [0]:
#Explore how many restaurants are missing from data generated from Tony's FB Search
fbdf = pd.read_csv("processsedTorontoFB.csv")

In [0]:
fbdf.head(5)

,Unnamed: 0,name,checkins,overall_star_rating,price_range,rating_count,engagement.count,restaurant_services.catering,restaurant_services.delivery,restaurant_services.groups,restaurant_services.kids,restaurant_services.outdoor,restaurant_services.reserve,restaurant_services.takeout,restaurant_services.waiter,restaurant_services.walkins,parking.lot,parking.street,parking.valet,payment_options.cash_only,payment_options.discover,payment_options.mastercard,payment_options.visa,restaurant_specialties.coffee,restaurant_specialties.dinner,restaurant_specialties.drinks,restaurant_specialties.lunch,id,engagement.social_sentence,payment_options.amex,restaurant_services.pickup,restaurant_specialties.breakfast
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Mi Mi Restaurant,1252.0,4.7,$$,15.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.109586e+14,88 people like this.,0.0,NaN,NaN
2,2,Tim Hortons,43.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.212788e+14,Be the first of your friends to like this.,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see what percentage of each feature is missing values/null

In [0]:
fbdf.isnull().mean() * 100

Unnamed: 0                           0.000000
name                                43.000346
checkins                            43.000346
overall_star_rating                 74.063832
price_range                         75.492568
rating_count                        43.645581
engagement.count                    43.000346
restaurant_services.catering        90.448208
restaurant_services.delivery        90.137113
restaurant_services.groups          90.448208
restaurant_services.kids            90.448208
restaurant_services.outdoor         90.448208
restaurant_services.reserve         89.445789
restaurant_services.takeout         90.137113
restaurant_services.waiter          90.448208
restaurant_services.walkins         90.448208
parking.lot                         75.239083
parking.street                      75.239083
parking.valet                       75.239083
payment_options.cash_only           91.369973
payment_options.discover            91.369973
payment_options.mastercard        

Only take features with 57% or more of their data


In [0]:
fb_subset = fbdf[['name','checkins', 'rating_count','engagement.count']]

In [0]:
fb_subset.head(5)

,fbcheckins,rating_count,engagement.count
0,NaN,NaN,NaN
1,1252.0,15.0,88.0
2,43.0,0.0,0.0
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [0]:
fb_subset= fb_subset.rename(columns={"checkins": "fbcheckins"})

remove NAmes column, have that already from Yelp

In [0]:
fb_subset = fb_subset.drop(fb_subset.columns[[0]], axis=1)

In [0]:
fb_subset.head(5)

,fbcheckins,rating_count,engagement.count
0,NaN,NaN,NaN
1,1252.0,15.0,88.0
2,43.0,0.0,0.0
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [0]:
# Load Yelp data
yelpdf = pd.read_csv("restaurant_toronto.csv")

Check that they both have same length, before appending them adjacently

In [0]:
len(fb_subset)

8679

In [0]:
len(yelpdf)

8679

In [0]:
yelpdf.head(5) ## Mi MI Restaurant and Tim Hortosn are 2nd and 3rd row just like FB df above 

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.BusinessAcceptsCreditCards,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.ByAppointmentOnly,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,hours,attributes,attributes.DogsAllowed,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Caters,attributes.WheelchairAccessible,attributes.AcceptsInsurance,attributes.RestaurantsTableService,attributes.Ambience,attributes.GoodForMeal,attributes.HappyHour,attributes.BusinessAcceptsBitcoin,attributes.BYOB,attributes.Corkage,attributes.GoodForDancing,attributes.CoatCheck,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours
0,0,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,Toronto,ON,M8Z 5G9,43.633291,-79.531768,3.0,13,0,"Restaurants, Burgers, Food",NaN,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,u'average',False,True,False,u'none',False,False,False,NaN,NaN,NaN,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,ON,M4M 1Y3,43.666376,-79.348773,4.0,116,1,"Vietnamese, Restaurants",NaN,True,True,"{'garage': False, 'street': True, 'validated':...",NaN,1,11:0-22:0,11:0-22:0,12:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,NaN,NaN,NaN,u'no',u'casual',True,u'average',False,True,True,u'beer_and_wine',False,False,False,NaN,NaN,True,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,ON,M5H 3V9,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",NaN,False,True,"{'garage': False, 'street': False, 'validated'...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no','casual',True,NaN,False,True,True,'none',False,False,NaN,NaN,NaN,NaN,"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,3-1265 York Mills Road,Toronto,ON,M3A 1Z3,43.765279,-79.326248,3.0,11,1,"Fast Food, Restaurants, Chicken Shop",NaN,True,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,'loud',False,True,True,'none',True,True,True,NaN,NaN,False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
4,4,NLaK58WvlNQdUunSIkt-jA,Zav Coffee Shop & Gallery,2048 Danforth Avenue,Toronto,ON,M4C 1J6,43.685608,-79.313936,4.5,24,1,"Coffee & Tea, Restaurants, Sandwiches, Food",NaN,True,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,1,0:0-0:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,8:30-17:0,8:30-17:0,NaN,NaN,False,u'free',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
mergedf = pd.concat([yelpdf, fb_subset], axis=1)

In [0]:
mergedf.head(5)

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.BusinessAcceptsCreditCards,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.ByAppointmentOnly,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,hours,attributes,attributes.DogsAllowed,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Caters,attributes.WheelchairAccessible,attributes.AcceptsInsurance,attributes.RestaurantsTableService,attributes.Ambience,attributes.GoodForMeal,attributes.HappyHour,attributes.BusinessAcceptsBitcoin,attributes.BYOB,attributes.Corkage,attributes.GoodForDancing,attributes.CoatCheck,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours,fbcheckins,rating_count,engagement.count
0,0,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,Toronto,ON,M8Z 5G9,43.633291,-79.531768,3.0,13,0,"Restaurants, Burgers, Food",NaN,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,u'average',False,True,False,u'none',False,False,False,NaN,NaN,NaN,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,ON,M4M 1Y3,43.666376,-79.348773,4.0,116,1,"Vietnamese, Restaurants",NaN,True,True,"{'garage': False, 'street': True, 'validated':...",NaN,1,11:0-22:0,11:0-22:0,12:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,NaN,NaN,NaN,u'no',u'casual',True,u'average',False,True,True,u'beer_and_wine',False,False,False,NaN,NaN,True,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1252.0,15.0,88.0
2,2,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,ON,M5H 3V9,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",NaN,False,True,"{'garage': False, 'street': False, 'validated'...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no','casual',True,NaN,False,True,True,'none',False,False,NaN,NaN,NaN,NaN,"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,0.0,0.0
3,3,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,3-1265 York Mills Road,Toronto,ON,M3A 1Z3,43.765279,-79.326248,3.0,11,1,"Fast Food, Restaurants, Chicken Shop",NaN,True,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,'loud',False,True,True,'none',True,True,True,NaN,NaN,False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NLaK58WvlNQdUunSIkt-jA,Zav Coffee Shop & Gallery,2048 Danforth Avenue,Toronto,ON,M4C 1J6,43.685608,-79.313936,4.5,24,1,"Coffee & Tea, Restaurants, Sandwiches, Food",NaN,True,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,1,0:0-0:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,8:30-17:0,8:30-17:0,NaN,NaN,False,u'free',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# drop first column with indices
mergedf = mergedf.drop(mergedf.columns[[0]], axis=1)

In [0]:
pd.options.display.max_columns = None
display(mergedf)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.BusinessAcceptsCreditCards,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.ByAppointmentOnly,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,hours,attributes,attributes.DogsAllowed,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Caters,attributes.WheelchairAccessible,attributes.AcceptsInsurance,attributes.RestaurantsTableService,attributes.Ambience,attributes.GoodForMeal,attributes.HappyHour,attributes.BusinessAcceptsBitcoin,attributes.BYOB,attributes.Corkage,attributes.GoodForDancing,attributes.CoatCheck,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours,fbcheckins,rating_count,engagement.count
0,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,Toronto,ON,M8Z 5G9,43.633291,-79.531768,3.0,13,0,"Restaurants, Burgers, Food",NaN,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,u'average',False,True,False,u'none',False,False,False,NaN,NaN,NaN,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,ON,M4M 1Y3,43.666376,-79.348773,4.0,116,1,"Vietnamese, Restaurants",NaN,True,True,"{'garage': False, 'street': True, 'validated':...",NaN,1,11:0-22:0,11:0-22:0,12:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,NaN,NaN,NaN,u'no',u'casual',True,u'average',False,True,True,u'beer_and_wine',False,False,False,NaN,NaN,True,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1252.0,15.0,88.0
2,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,ON,M5H 3V9,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",NaN,False,True,"{'garage': False, 'street': False, 'validated'...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no','casual',True,NaN,False,True,True,'none',False,False,NaN,NaN,NaN,NaN,"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,0.0,0.0
3,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,3-1265 York Mills Road,Toronto,ON,M3A 1Z3,43.765279,-79.326248,3.0,11,1,"Fast Food, Restaurants, Chicken Shop",NaN,True,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,'loud',False,True,True,'none',True,True,True,NaN,NaN,False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NLaK58WvlNQdUunSIkt-jA,Zav Coffee Shop & Gallery,2048 Danforth Avenue,Toronto,ON,M4C 1J6,43.685608,-79.313936,4.5,24,1,"Coffee & Tea, Restaurants, Sandwiches, Food",NaN,True,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,1,0:0-0:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,8:30-17:0,8:30-17:0,NaN,NaN,False,u'free',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [0]:
mergedf.to_csv("Final_merged_FB_Yelp_v1.csv")

In [0]:
pwd

'/content/drive/Shared drives/Data Science Principles Project'

In [0]:
#see what percentage is null again for features
type(mergedf.isnull().mean() * 100)

pandas.core.series.Series

In [0]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    yelpfbcolumns = mergedf.isnull().mean() * 100
    print(mergedf.isnull().mean() * 100)

business_id                                0.000000
name                                       0.000000
address                                    0.276530
city                                       0.000000
state                                      0.000000
postal_code                                0.230441
latitude                                   0.000000
longitude                                  0.000000
stars                                      0.000000
review_count                               0.000000
is_open                                    0.000000
categories                                 0.000000
attributes.BusinessAcceptsCreditCards     96.865998
attributes.BikeParking                    39.002189
attributes.GoodForKids                    22.456504
attributes.BusinessParking                18.780966
attributes.ByAppointmentOnly              96.739256
attributes.RestaurantsPriceRange2         19.322503
hours.Monday                              32.526789
hours.Tuesda

Drop all columns with missing values more than 44% of their data

In [0]:
df = mergedf[mergedf.columns[mergedf.isnull().mean() < .44]]

In [0]:
df.isnull().mean() * 100

business_id                             0.000000
name                                    0.000000
address                                 0.276530
city                                    0.000000
state                                   0.000000
postal_code                             0.230441
latitude                                0.000000
longitude                               0.000000
stars                                   0.000000
review_count                            0.000000
is_open                                 0.000000
categories                              0.000000
attributes.BikeParking                 39.002189
attributes.GoodForKids                 22.456504
attributes.BusinessParking             18.780966
attributes.RestaurantsPriceRange2      19.322503
hours.Monday                           32.526789
hours.Tuesday                          26.143565
hours.Wednesday                        23.919806
hours.Thursday                         23.067174
hours.Friday        

Drop useless columns for prediction : name, city, state, address, postal code, bid

In [0]:
#drop name and bid cause it shouldn't help with score, and city and state are same for Toronto
df1 = df.drop(['name', 'city', 'state', 'address', 'postal_code'], axis=1)

In [0]:
df1.head(5)

,business_id,latitude,longitude,stars,review_count,is_open,categories,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Ambience,fbcheckins,rating_count,engagement.count
0,rVBPQdeayMYht4Uv_FOLHg,43.633291,-79.531768,3.0,13,0,"Restaurants, Burgers, Food",NaN,True,"{'garage': False, 'street': False, 'validated'...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,u'average',False,True,False,u'none',False,False,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN
1,0QjROMVW9ACKjhSEfHqNCQ,43.666376,-79.348773,4.0,116,1,"Vietnamese, Restaurants",True,True,"{'garage': False, 'street': True, 'validated':...",1,11:0-22:0,11:0-22:0,12:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,u'no',u'casual',True,u'average',False,True,True,u'beer_and_wine',False,False,"{'romantic': False, 'intimate': False, 'classy...",1252.0,15.0,88.0
2,8k62wYhDVq1-652YbJi5eg,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",False,True,"{'garage': False, 'street': False, 'validated'...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no','casual',True,NaN,False,True,True,'none',False,False,"{'touristy': False, 'hipster': False, 'romanti...",43.0,0.0,0.0
3,0DnQh8SE8BSnvJltGCCiWg,43.765279,-79.326248,3.0,11,1,"Fast Food, Restaurants, Chicken Shop",True,True,"{'garage': False, 'street': False, 'validated'...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,'loud',False,True,True,'none',True,True,"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN
4,NLaK58WvlNQdUunSIkt-jA,43.685608,-79.313936,4.5,24,1,"Coffee & Tea, Restaurants, Sandwiches, Food",True,NaN,"{'garage': False, 'street': False, 'validated'...",1,0:0-0:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,8:30-17:0,8:30-17:0,u'free',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN


Add in Sent analysis columns

In [0]:
sadf = pd.read_csv("Yelp_Filtered_SA_Dataset.csv")

In [0]:
sadf.head(25)

,Unnamed: 0,business_id,latitude,longitude,review_count,stars,is_open,pos_reviews
0,2,007Dg4ESDVacWcC4Vq704Q,43.603232,-79.538424,9,4.0,0,7
1,3,rVBPQdeayMYht4Uv_FOLHg,43.633291,-79.531768,13,3.0,0,11
2,5,0QjROMVW9ACKjhSEfHqNCQ,43.666376,-79.348773,116,4.0,1,109
3,10,GtQPbazps0jt7_cJghfmsA,43.670936,-79.384643,42,2.5,1,26
4,11,8k62wYhDVq1-652YbJi5eg,43.649859,-79.382060,8,3.0,1,7
5,12,PYLv9n0y8MimzTK2zCBoFw,43.664707,-79.413939,4,3.0,1,1
6,14,0DnQh8SE8BSnvJltGCCiWg,43.765279,-79.326248,11,3.0,1,8
7,15,NLaK58WvlNQdUunSIkt-jA,43.685608,-79.313936,24,4.5,1,24
8,16,OIKp-6ObAd-yW-32U3c22Q,43.664927,-79.411553,9,4.0,0,7
9,17,cicPsia8Wj-DNRkmLbD_xg,43.705842,-79.397841,100,3.5,1,79


In [0]:
#check if same length
len(sadf)

11135

In [0]:
sadf = sadf[]

 Merge on Business ID

In [0]:
Samergedf0 = pd.merge(df1, sadf, on='business_id', how='left')

In [0]:
#Just made so I could doublecheck with original SA.csv to see if it was successful in matching business IDs
view  = Samergedf0[['business_id','pos_reviews']]

In [0]:
view.head(50)

,business_id,pos_reviews
0,rVBPQdeayMYht4Uv_FOLHg,11
1,0QjROMVW9ACKjhSEfHqNCQ,109
2,8k62wYhDVq1-652YbJi5eg,7
3,0DnQh8SE8BSnvJltGCCiWg,8
4,NLaK58WvlNQdUunSIkt-jA,24
5,cicPsia8Wj-DNRkmLbD_xg,79
6,0nqQIM2PvOcTX6v0L537_Q,17
7,xVXyrTWbG8U3szze-aA7eg,14
8,r3jpH6ZuCQY3o1yBuJ67PQ,12
9,TKKX9bllRBMzp0NYr441PQ,50


In [0]:
len(view)

8679

In [0]:
#All Reviews present
view.isnull().mean() * 100

business_id    0.0
pos_reviews    0.0
dtype: float64

In [0]:
#df2 is new merged df with fb + yelp + sentiment
df2 = Samergedf0

In [0]:
df2.head(5)

,business_id,latitude_x,longitude_x,stars_x,review_count_x,is_open_x,categories,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Ambience,fbcheckins,rating_count,engagement.count,Unnamed: 0,latitude_y,longitude_y,review_count_y,stars_y,is_open_y,pos_reviews
0,rVBPQdeayMYht4Uv_FOLHg,43.633291,-79.531768,3.0,13,0,"Restaurants, Burgers, Food",NaN,True,"{'garage': False, 'street': False, 'validated'...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,u'average',False,True,False,u'none',False,False,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,3,43.633291,-79.531768,13,3.0,0,11
1,0QjROMVW9ACKjhSEfHqNCQ,43.666376,-79.348773,4.0,116,1,"Vietnamese, Restaurants",True,True,"{'garage': False, 'street': True, 'validated':...",1,11:0-22:0,11:0-22:0,12:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,u'no',u'casual',True,u'average',False,True,True,u'beer_and_wine',False,False,"{'romantic': False, 'intimate': False, 'classy...",1252.0,15.0,88.0,5,43.666376,-79.348773,116,4.0,1,109
2,8k62wYhDVq1-652YbJi5eg,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",False,True,"{'garage': False, 'street': False, 'validated'...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no','casual',True,NaN,False,True,True,'none',False,False,"{'touristy': False, 'hipster': False, 'romanti...",43.0,0.0,0.0,11,43.649859,-79.382060,8,3.0,1,7
3,0DnQh8SE8BSnvJltGCCiWg,43.765279,-79.326248,3.0,11,1,"Fast Food, Restaurants, Chicken Shop",True,True,"{'garage': False, 'street': False, 'validated'...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,'loud',False,True,True,'none',True,True,"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,14,43.765279,-79.326248,11,3.0,1,8
4,NLaK58WvlNQdUunSIkt-jA,43.685608,-79.313936,4.5,24,1,"Coffee & Tea, Restaurants, Sandwiches, Food",True,NaN,"{'garage': False, 'street': False, 'validated'...",1,0:0-0:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,8:30-17:0,8:30-17:0,u'free',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,15,43.685608,-79.313936,24,4.5,1,24


In [0]:
df2.to_csv("Final_merged_FB_Yelp_v0.csv")

In [0]:
df2.isnull().mean() * 100

business_id                             0.000000
latitude_x                              0.000000
longitude_x                             0.000000
stars_x                                 0.000000
review_count_x                          0.000000
is_open_x                               0.000000
categories                              0.000000
attributes.BikeParking                 39.002189
attributes.GoodForKids                 22.456504
attributes.BusinessParking             18.780966
attributes.RestaurantsPriceRange2      19.322503
hours.Monday                           32.526789
hours.Tuesday                          26.143565
hours.Wednesday                        23.919806
hours.Thursday                         23.067174
hours.Friday                           22.951953
hours.Saturday                         26.546837
hours.Sunday                           35.971886
attributes.WiFi                        33.183546
attributes.RestaurantsAttire           26.397050
attributes.Restauran

In [0]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', -1)
df2['attributes.Ambience'].head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


0    {'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}
1    {'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}
2    {'touristy': False, 'hipster': False, 'romantic': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}               
3    {'touristy': False, 'hipster': False, 'romantic': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}               
4    NaN                                                                                                                                                            
Name: attributes.Ambience, dtype: object

In [0]:
temp = df2['categories'].str.split(',', expand = True)

In [0]:
temp.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Restaurants,Burgers,Food,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Vietnamese,Restaurants,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Bagels,Donuts,Food,Cafes,Coffee & Tea,Restaurants,Bakeries,None,None,None,None,None,None,None,None,None,None
3,Fast Food,Restaurants,Chicken Shop,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Coffee & Tea,Restaurants,Sandwiches,Food,None,None,None,None,None,None,None,None,None,None,None,None,None


In [0]:
#fix Ambience into JSON ready format
df2['attributes.Ambience'] = df2['attributes.Ambience'].str.replace("\'", "\"").str.replace("False", "\"False\"").str.replace("True", "\"True\"")

In [0]:
df2['attributes.Ambience'] = df2['attributes.Ambience'].str.replace("None", "\"False\"")

In [0]:
df2.head(5)

,business_id,latitude_x,longitude_x,stars_x,review_count_x,is_open_x,categories,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Ambience,fbcheckins,rating_count,engagement.count,Unnamed: 0,latitude_y,longitude_y,review_count_y,stars_y,is_open_y,pos_reviews
0,rVBPQdeayMYht4Uv_FOLHg,43.633291,-79.531768,3.0,13,0,"Restaurants, Burgers, Food",NaN,True,"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,u'average',False,True,False,u'none',False,False,"{""romantic"": ""False"", ""intimate"": ""False"", ""classy"": ""False"", ""hipster"": ""False"", ""divey"": ""False"", ""touristy"": ""False"", ""trendy"": ""False"", ""upscale"": ""False"", ""casual"": ""True""}",NaN,NaN,NaN,3,43.633291,-79.531768,13,3.0,0,11
1,0QjROMVW9ACKjhSEfHqNCQ,43.666376,-79.348773,4.0,116,1,"Vietnamese, Restaurants",True,True,"{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",1,11:0-22:0,11:0-22:0,12:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,u'no',u'casual',True,u'average',False,True,True,u'beer_and_wine',False,False,"{""romantic"": ""False"", ""intimate"": ""False"", ""classy"": ""False"", ""hipster"": ""False"", ""divey"": ""False"", ""touristy"": ""False"", ""trendy"": ""False"", ""upscale"": ""False"", ""casual"": ""True""}",1252.0,15.0,88.0,5,43.666376,-79.348773,116,4.0,1,109
2,8k62wYhDVq1-652YbJi5eg,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Restaurants, Bakeries",False,True,"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no','casual',True,NaN,False,True,True,'none',False,False,"{""touristy"": ""False"", ""hipster"": ""False"", ""romantic"": ""False"", ""intimate"": ""False"", ""trendy"": ""False"", ""upscale"": ""False"", ""classy"": ""False"", ""casual"": ""False""}",43.0,0.0,0.0,11,43.649859,-79.382060,8,3.0,1,7
3,0DnQh8SE8BSnvJltGCCiWg,43.765279,-79.326248,3.0,11,1,"Fast Food, Restaurants, Chicken Shop",True,True,"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,'loud',False,True,True,'none',True,True,"{""touristy"": ""False"", ""hipster"": ""False"", ""romantic"": ""False"", ""intimate"": ""False"", ""trendy"": ""False"", ""upscale"": ""False"", ""classy"": ""False"", ""casual"": ""False""}",NaN,NaN,NaN,14,43.765279,-79.326248,11,3.0,1,8
4,NLaK58WvlNQdUunSIkt-jA,43.685608,-79.313936,4.5,24,1,"Coffee & Tea, Restaurants, Sandwiches, Food",True,NaN,"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",1,0:0-0:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,8:30-17:0,8:30-17:0,u'free',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,15,43.685608,-79.313936,24,4.5,1,24


In [0]:
amb_list = df2['attributes.Ambience'].tolist()
filler = {'romantic': 'False', 'intimate': 'False', 'classy': 'False', 'hipster': 'False', 'divey': 'False', 'touristy': 'False', 'trendy': 'False', 'upscale': 'False', 'casual': 'False'}

#print(amb_list[0])
d =[]
for i in amb_list:
  try:
    if json.loads(i) =='False':
      d.append(filler)
    else:
      d.append(json.loads(i))
  except:
    d.append(filler)
    #do nothing
#print(d[0])
#test = [ind for ind in range(len(d)) if isinstance(d[ind],str)]
#print(test)
#print(d[147],"\n",d[262],"\n", d[2244],"\n",d[5814],"\n")
ls = [item for item in d if isinstance(item, str)]
#print(ls)
temp = pd.json_normalize(d).add_prefix('ambience.')
#print(d.head(5))
print(len(temp))
print(len(df2))
temp.head(5)

8679
8679


,ambience.romantic,ambience.intimate,ambience.classy,ambience.hipster,ambience.divey,ambience.touristy,ambience.trendy,ambience.upscale,ambience.casual
0,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,True
2,False,False,False,False,NaN,False,False,False,False
3,False,False,False,False,NaN,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [0]:
# add ambiance columns now that they're seperated have to drop 'ambience later
df3 = pd.concat([df2, temp], axis=1)

In [0]:
categories = df3['categories'].str.get_dummies(sep=', ').add_prefix('cat_')

In [0]:
categories.head(5)

,cat_Acai Bowls,cat_Accessories,cat_Accountants,cat_Acne Treatment,cat_Active Life,cat_Adult Entertainment,cat_Afghan,cat_African,cat_Airport Lounges,cat_Alternative Medicine,cat_American (New),cat_American (Traditional),cat_Amusement Parks,cat_Animal Shelters,cat_Antiques,cat_Appliances,cat_Arabian,cat_Arcades,cat_Argentine,cat_Art Classes,cat_Art Galleries,cat_Art Schools,cat_Arts & Crafts,cat_Arts & Entertainment,cat_Asian Fusion,cat_Australian,cat_Austrian,cat_Automotive,cat_Bagels,cat_Bakeries,cat_Bangladeshi,cat_Bankruptcy Law,cat_Barbeque,cat_Bars,cat_Bartenders,cat_Basque,cat_Beaches,cat_Beauty & Spas,cat_Bed & Breakfast,cat_Beer,cat_Beer Bar,cat_Beer Hall,cat_Belgian,cat_Bistros,cat_Bocce Ball,cat_Books,cat_Bookstores,cat_Bowling,cat_Brasseries,cat_Brazilian,cat_Breakfast & Brunch,cat_Breweries,cat_Brewpubs,cat_British,cat_Bubble Tea,cat_Buffets,cat_Burgers,cat_Burmese,cat_Butcher,cat_Cabaret,cat_Cafes,cat_Cafeteria,cat_Cajun/Creole,cat_Cambodian,cat_Canadian (New),cat_Candy Stores,cat_Cannabis Clinics,cat_Cantonese,cat_Cards & Stationery,cat_Caribbean,cat_Casinos,cat_Caterers,cat_Champagne Bars,cat_Cheese Shops,cat_Cheesesteaks,cat_Chicken Shop,cat_Chicken Wings,cat_Chinese,cat_Chocolatiers & Shops,cat_Churches,cat_Cideries,cat_Cinema,cat_Cocktail Bars,cat_Coffee & Tea,cat_Coffee & Tea Supplies,cat_Coffee Roasteries,cat_Colombian,cat_Comedy Clubs,cat_Comfort Food,cat_Comic Books,cat_Community Centers,cat_Community Service/Non-Profit,cat_Convenience Stores,cat_Cooking Classes,cat_Cooking Schools,cat_Couriers & Delivery Services,cat_Creperies,cat_Cuban,cat_Custom Cakes,cat_Customized Merchandise,cat_Czech,cat_Czech/Slovakian,cat_Dance Clubs,cat_Day Spas,cat_Delicatessen,cat_Delis,cat_Department Stores,cat_Desserts,cat_Dim Sum,cat_Diners,cat_Dinner Theater,cat_Discount Store,cat_Dive Bars,cat_Do-It-Yourself Food,cat_Dog Walkers,cat_Donairs,cat_Donuts,cat_Dry Cleaning & Laundry,cat_Dumplings,cat_Education,cat_Egyptian,cat_Electricians,cat_Electronics,cat_Escape Games,cat_Ethical Grocery,cat_Ethiopian,cat_Ethnic Food,cat_Ethnic Grocery,cat_Event Planning & Services,cat_Eyelash Service,cat_Falafel,cat_Farmers Market,cat_Fashion,cat_Fast Food,cat_Festivals,cat_Filipino,cat_Financial Services,cat_Fish & Chips,cat_Fitness & Instruction,cat_Florists,cat_Flowers & Gifts,cat_Fondue,cat_Food,cat_Food Court,cat_Food Delivery Services,cat_Food Stands,cat_Food Tours,cat_Food Trucks,cat_French,cat_Fruits & Veggies,cat_Furniture Stores,cat_Gas Stations,cat_Gastropubs,cat_Gay Bars,cat_Gelato,cat_German,cat_Gift Shops,cat_Gluten-Free,cat_Greek,cat_Grocery,cat_Hainan,cat_Hair Loss Centers,cat_Hair Removal,cat_Hair Salons,cat_Haitian,cat_Hakka,cat_Halal,cat_Hawaiian,cat_Head Shops,cat_Health & Medical,cat_Health Markets,cat_Himalayan/Nepalese,cat_Hobby Shops,cat_Home & Garden,cat_Home Cleaning,cat_Home Decor,cat_Home Services,cat_Hong Kong Style Cafe,cat_Hookah Bars,cat_Hostels,cat_Hot Dogs,cat_Hot Pot,cat_Hotels,cat_Hotels & Travel,cat_Hungarian,cat_IT Services & Computer Repair,cat_Iberian,cat_Ice Cream & Frozen Yogurt,cat_Imported Food,cat_Indian,cat_Indonesian,cat_Indoor Playcentre,cat_International,cat_International Grocery,cat_Internet Cafes,cat_Investing,cat_Irish,cat_Irish Pub,cat_Italian,cat_Izakaya,cat_Japanese,cat_Jazz & Blues,cat_Jewelry,cat_Juice Bars & Smoothies,cat_Karaoke,cat_Kebab,cat_Kombucha,cat_Korean,cat_Kosher,cat_Landmarks & Historical Buildings,cat_Laotian,cat_Laser Hair Removal,cat_Latin American,cat_Laundromat,cat_Laundry Services,cat_Lawyers,cat_Lebanese,cat_Life Coach,cat_Live/Raw Food,cat_Local Flavor,cat_Local Services,cat_Lounges,cat_Macarons,cat_Mags,cat_Malaysian,cat_Marketing,cat_Massage,cat_Meat Shops,cat_Medical Spas,cat_Meditation Centers,cat_Mediterranean,cat_Men's Clothing,cat_Mexican,cat_Middle Eastern,cat_Milkshake Bars,cat_Minho,cat_Mobile Phone Repair,cat_Modern European,cat_Mongolian,cat_Moroccan,cat_Mosques,cat_Music & DVDs,cat_Music & Video,cat_Music Venues,cat_Musical Instruments & Teachers,c

In [0]:
print(len(categories))

8679


In [0]:
#business parking
df3['attributes.BusinessParking'] = df3['attributes.BusinessParking'].str.replace("\'", "\"").str.replace("False", "\"False\"").str.replace("True", "\"True\"")
df3['attributes.BusinessParking'] = df3['attributes.BusinessParking'].str.replace("None", "\"False\"")

In [0]:
bp_list = df3['attributes.BusinessParking'].tolist()
filler = {'garage': 'False', 'street': 'False', 'validated': 'False', 'lot': 'False', 'valet': 'False'}

#print(amb_list[0])
e = []
for i in bp_list:
  try:
    if json.loads(i) =='False':
      e.append(filler)
    else:
      e.append(json.loads(i))
  except:
    e.append(filler)

temp2 = pd.json_normalize(e).add_prefix('business_parking.')
print(temp2.head(5))
#print(len(temp2))

  business_parking.garage business_parking.street business_parking.validated  \
0  False                   False                   False                       
1  False                   True                    False                       
2  False                   False                   False                       
3  False                   False                   False                       
4  False                   False                   False                       

  business_parking.lot business_parking.valet  
0  True                 False                  
1  False                False                  
2  False                False                  
3  False                False                  
4  False                False                  


In [0]:
#Add categories, business parking, and drop those columns
df4 = pd.concat([df3, categories], axis=1)
df4 = pd.concat([df4, temp2], axis=1)
df4 = df4.drop(['cat_Restaurants', 'attributes.BusinessParking', 'attributes.Ambience', 'categories'], axis=1)

In [0]:
df4.head(5)

,business_id,latitude_x,longitude_x,stars_x,review_count_x,is_open_x,attributes.BikeParking,attributes.GoodForKids,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,fbcheckins,rating_count,engagement.count,Unnamed: 0,latitude_y,longitude_y,review_count_y,stars_y,is_open_y,pos_reviews,ambience.romantic,ambience.intimate,ambience.classy,ambience.hipster,ambience.divey,ambience.touristy,ambience.trendy,ambience.upscale,ambience.casual,cat_Acai Bowls,cat_Accessories,cat_Accountants,cat_Acne Treatment,cat_Active Life,cat_Adult Entertainment,cat_Afghan,cat_African,cat_Airport Lounges,cat_Alternative Medicine,cat_American (New),cat_American (Traditional),cat_Amusement Parks,cat_Animal Shelters,cat_Antiques,cat_Appliances,cat_Arabian,cat_Arcades,cat_Argentine,cat_Art Classes,cat_Art Galleries,cat_Art Schools,cat_Arts & Crafts,cat_Arts & Entertainment,cat_Asian Fusion,cat_Australian,cat_Austrian,cat_Automotive,cat_Bagels,cat_Bakeries,cat_Bangladeshi,cat_Bankruptcy Law,cat_Barbeque,cat_Bars,cat_Bartenders,cat_Basque,cat_Beaches,cat_Beauty & Spas,cat_Bed & Breakfast,cat_Beer,cat_Beer Bar,cat_Beer Hall,cat_Belgian,cat_Bistros,cat_Bocce Ball,cat_Books,cat_Bookstores,cat_Bowling,cat_Brasseries,cat_Brazilian,cat_Breakfast & Brunch,cat_Breweries,cat_Brewpubs,cat_British,cat_Bubble Tea,cat_Buffets,cat_Burgers,cat_Burmese,cat_Butcher,cat_Cabaret,cat_Cafes,cat_Cafeteria,cat_Cajun/Creole,cat_Cambodian,cat_Canadian (New),cat_Candy Stores,cat_Cannabis Clinics,cat_Cantonese,cat_Cards & Stationery,cat_Caribbean,cat_Casinos,cat_Caterers,cat_Champagne Bars,cat_Cheese Shops,cat_Cheesesteaks,cat_Chicken Shop,cat_Chicken Wings,cat_Chinese,cat_Chocolatiers & Shops,cat_Churches,cat_Cideries,cat_Cinema,cat_Cocktail Bars,cat_Coffee & Tea,cat_Coffee & Tea Supplies,cat_Coffee Roasteries,cat_Colombian,cat_Comedy Clubs,cat_Comfort Food,cat_Comic Books,cat_Community Centers,cat_Community Service/Non-Profit,cat_Convenience Stores,cat_Cooking Classes,cat_Cooking Schools,cat_Couriers & Delivery Services,cat_Creperies,cat_Cuban,cat_Custom Cakes,cat_Customized Merchandise,cat_Czech,cat_Czech/Slovakian,cat_Dance Clubs,cat_Day Spas,cat_Delicatessen,cat_Delis,cat_Department Stores,cat_Desserts,cat_Dim Sum,cat_Diners,cat_Dinner Theater,cat_Discount Store,cat_Dive Bars,cat_Do-It-Yourself Food,cat_Dog Walkers,cat_Donairs,cat_Donuts,cat_Dry Cleaning & Laundry,cat_Dumplings,cat_Education,cat_Egyptian,cat_Electricians,cat_Electronics,cat_Escape Games,cat_Ethical Grocery,cat_Ethiopian,cat_Ethnic Food,cat_Ethnic Grocery,cat_Event Planning & Services,cat_Eyelash Service,cat_Falafel,cat_Farmers Market,cat_Fashion,cat_Fast Food,cat_Festivals,cat_Filipino,cat_Financial Services,cat_Fish & Chips,cat_Fitness & Instruction,cat_Florists,cat_Flowers & Gifts,cat_Fondue,cat_Food,cat_Food Court,cat_Food Delivery Services,cat_Food Stands,cat_Food Tours,cat_Food Trucks,cat_French,cat_Fruits & Veggies,cat_Furniture Stores,cat_Gas Stations,cat_Gastropubs,cat_Gay Bars,cat_Gelato,cat_German,cat_Gift Shops,cat_Gluten-Free,cat_Greek,cat_Grocery,cat_Hainan,cat_Hair Loss Centers,cat_Hair Removal,cat_Hair Salons,cat_Haitian,cat_Hakka,cat_Halal,cat_Hawaiian,cat_Head Shops,cat_Health & Medical,cat_Health Markets,cat_Himalayan/Nepalese,cat_Hobby Shops,cat_Home & Garden,cat_Home Cleaning,cat_Home Decor,cat_Home Services,cat_Hong Kong Style Cafe,cat_Hookah Bars,cat_Hostels,cat_Hot Dogs,cat_Hot Pot,cat_Hotels,cat_Hotels & Travel,cat_Hungarian,cat_IT Services & Computer Repair,cat_Iberian,cat_Ice Cream & Frozen Yogurt,cat_Imported Food,cat_Indian,cat_Indonesian,cat_Indoor Playcentre,cat_International,cat_International Grocery,cat_Internet Cafes,cat_Investing,cat_Irish,cat

In [0]:
pwd

'/content/drive/Shared drives/Data Science Principles Project'

In [0]:
#Unnamed: 0	latitude_y	longitude_y	review_count_y	stars_y	is_open_y
df4 = df4.drop(['Unnamed: 0', 'latitude_y', 'longitude_y', 'review_count_y','stars_y','is_open_y'], axis=1)

In [0]:
df4 = df4.rename(columns={"latitude_x": "latitude", "longitude_x": "longitude","stars_x":"stars","review_count_x":"review_count","is_open_x":"is_open"})

export merged code

In [0]:
df4.to_csv("alldone_1hot.csv")

Now to handle hours

In [0]:
df4[['MonHoursOpen','MonHoursClose']] = df4['hours.Monday'].str.split('-',expand=True) 

In [0]:
df4[['MonHoursOpen','MonHoursClose']] 

,MonHoursOpen,MonHoursClose
0,NaN,NaN
1,11:0,22:0
2,NaN,NaN
3,NaN,NaN
4,0:0,0:0
...,...,...
8674,12:0,23:0
8675,11:0,23:0
8676,11:30,22:0
8677,11:0,21:0


In [0]:
tri = df4['MonHoursOpen'].astype(str)

In [0]:
print(tri)

0       nan 
1       1100
2       nan 
3       nan 
4       0000
        ... 
8674    1200
8675    1100
8676    1130
8677    1100
8678    nan 
Name: MonHoursOpen, Length: 8679, dtype: object


In [0]:
#function to fix the time strings
def fixTime(timestr):
  x = timestr
  lis = x.split(":")
  new = []
  if(len(lis[0])==1):
    new.append('0'+ lis[0])
  else:
    new.append(lis[0])
  try:
    if(len(lis[1])==1):
      new.append(lis[1]+ '0')
    else:
      new.append(lis[1])
  except:
    x = 0 # do nothing
  new = ''.join(new)
  return new

print(fixTime("0:0"))
print(fixTime("10:30"))

0000
1030


In [0]:
tri = tri.apply(fixTime)

In [0]:
df4[['MonHoursOpen','MonHoursClose']] = df4['hours.Monday'].str.split('-',expand=True)
df4[['TuesHoursOpen','TuesHoursClose']] = df4['hours.Tuesday'].str.split('-',expand=True) 
df4[['WedHoursOpen','WedHoursClose']] = df4['hours.Wednesday'].str.split('-',expand=True) 
df4[['ThursHoursOpen','ThursHoursClose']] = df4['hours.Thursday'].str.split('-',expand=True) 
df4[['FriHoursOpen','FriHoursClose']] = df4['hours.Friday'].str.split('-',expand=True) 
df4[['SatHoursOpen','SatHoursClose']] = df4['hours.Saturday'].str.split('-',expand=True) 
df4[['SunHoursOpen','SunHoursClose']] = df4['hours.Sunday'].str.split('-',expand=True) 

In [0]:
df4.head(1)

,business_id,latitude,longitude,stars,review_count,is_open,attributes.BikeParking,attributes.GoodForKids,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,fbcheckins,rating_count,engagement.count,pos_reviews,ambience.romantic,ambience.intimate,ambience.classy,ambience.hipster,ambience.divey,ambience.touristy,ambience.trendy,ambience.upscale,ambience.casual,cat_Acai Bowls,cat_Accessories,cat_Accountants,cat_Acne Treatment,cat_Active Life,cat_Adult Entertainment,cat_Afghan,cat_African,cat_Airport Lounges,cat_Alternative Medicine,cat_American (New),cat_American (Traditional),cat_Amusement Parks,cat_Animal Shelters,cat_Antiques,cat_Appliances,cat_Arabian,cat_Arcades,cat_Argentine,cat_Art Classes,cat_Art Galleries,cat_Art Schools,cat_Arts & Crafts,cat_Arts & Entertainment,cat_Asian Fusion,cat_Australian,cat_Austrian,cat_Automotive,cat_Bagels,cat_Bakeries,cat_Bangladeshi,cat_Bankruptcy Law,cat_Barbeque,cat_Bars,cat_Bartenders,cat_Basque,cat_Beaches,cat_Beauty & Spas,cat_Bed & Breakfast,cat_Beer,cat_Beer Bar,cat_Beer Hall,cat_Belgian,cat_Bistros,cat_Bocce Ball,cat_Books,cat_Bookstores,cat_Bowling,cat_Brasseries,cat_Brazilian,cat_Breakfast & Brunch,cat_Breweries,cat_Brewpubs,cat_British,cat_Bubble Tea,cat_Buffets,cat_Burgers,cat_Burmese,cat_Butcher,cat_Cabaret,cat_Cafes,cat_Cafeteria,cat_Cajun/Creole,cat_Cambodian,cat_Canadian (New),cat_Candy Stores,cat_Cannabis Clinics,cat_Cantonese,cat_Cards & Stationery,cat_Caribbean,cat_Casinos,cat_Caterers,cat_Champagne Bars,cat_Cheese Shops,cat_Cheesesteaks,cat_Chicken Shop,cat_Chicken Wings,cat_Chinese,cat_Chocolatiers & Shops,cat_Churches,cat_Cideries,cat_Cinema,cat_Cocktail Bars,cat_Coffee & Tea,cat_Coffee & Tea Supplies,cat_Coffee Roasteries,cat_Colombian,cat_Comedy Clubs,cat_Comfort Food,cat_Comic Books,cat_Community Centers,cat_Community Service/Non-Profit,cat_Convenience Stores,cat_Cooking Classes,cat_Cooking Schools,cat_Couriers & Delivery Services,cat_Creperies,cat_Cuban,cat_Custom Cakes,cat_Customized Merchandise,cat_Czech,cat_Czech/Slovakian,cat_Dance Clubs,cat_Day Spas,cat_Delicatessen,cat_Delis,cat_Department Stores,cat_Desserts,cat_Dim Sum,cat_Diners,cat_Dinner Theater,cat_Discount Store,cat_Dive Bars,cat_Do-It-Yourself Food,cat_Dog Walkers,cat_Donairs,cat_Donuts,cat_Dry Cleaning & Laundry,cat_Dumplings,cat_Education,cat_Egyptian,cat_Electricians,cat_Electronics,cat_Escape Games,cat_Ethical Grocery,cat_Ethiopian,cat_Ethnic Food,cat_Ethnic Grocery,cat_Event Planning & Services,cat_Eyelash Service,cat_Falafel,cat_Farmers Market,cat_Fashion,cat_Fast Food,cat_Festivals,cat_Filipino,cat_Financial Services,cat_Fish & Chips,cat_Fitness & Instruction,cat_Florists,cat_Flowers & Gifts,cat_Fondue,cat_Food,cat_Food Court,cat_Food Delivery Services,cat_Food Stands,cat_Food Tours,cat_Food Trucks,cat_French,cat_Fruits & Veggies,cat_Furniture Stores,cat_Gas Stations,cat_Gastropubs,cat_Gay Bars,cat_Gelato,cat_German,cat_Gift Shops,cat_Gluten-Free,cat_Greek,cat_Grocery,cat_Hainan,cat_Hair Loss Centers,cat_Hair Removal,cat_Hair Salons,cat_Haitian,cat_Hakka,cat_Halal,cat_Hawaiian,cat_Head Shops,cat_Health & Medical,cat_Health Markets,cat_Himalayan/Nepalese,cat_Hobby Shops,cat_Home & Garden,cat_Home Cleaning,cat_Home Decor,cat_Home Services,cat_Hong Kong Style Cafe,cat_Hookah Bars,cat_Hostels,cat_Hot Dogs,cat_Hot Pot,cat_Hotels,cat_Hotels & Travel,cat_Hungarian,cat_IT Services & Computer Repair,cat_Iberian,cat_Ice Cream & Frozen Yogurt,cat_Imported Food,cat_Indian,cat_Indonesian,cat_Indoor Playcentre,cat_International,cat_International Grocery,cat_Internet Cafes,cat_Investing,cat_Irish,cat_Irish Pub,cat_Italian,cat_Izakaya,cat_Japanese,cat_Jazz & Blues,cat_Jewelry,

In [0]:
df4[['MonHoursOpen','MonHoursClose','TuesHoursOpen','TuesHoursClose','WedHoursOpen','WedHoursClose','ThursHoursOpen','ThursHoursClose','FriHoursOpen','FriHoursClose','SatHoursOpen','SatHoursClose','SunHoursOpen','SunHoursClose']] = df4[['MonHoursOpen','MonHoursClose','TuesHoursOpen','TuesHoursClose','WedHoursOpen','WedHoursClose','ThursHoursOpen','ThursHoursClose','FriHoursOpen','FriHoursClose','SatHoursOpen','SatHoursClose','SunHoursOpen','SunHoursClose']].apply(fixTime)

AttributeError: ignored

In [0]:
df4['MonHoursOpen'] = df4['MonHoursOpen'].apply(fixTime)
df4['MonHoursClose'] = df4['MonHoursClose'].apply(fixTime)
df4['TuesHoursOpen'] = df4['TuesHoursOpen'].apply(fixTime)
df4['TuesHoursClose'] = df4['TuesHoursClose'].apply(fixTime)
df4['WedHoursOpen'] = df4['WedHoursOpen'].apply(fixTime)
df4['WedHoursClose'] = df4['WedHoursClose'].apply(fixTime)
df4['ThursHoursOpen'] = df4['ThursHoursOpen'].apply(fixTime)
df4['ThursHoursClose'] = df4['ThursHoursClose'].apply(fixTime)
df4['FriHoursOpen'] = df4['FriHoursOpen'].apply(fixTime)
df4['FriHoursClose'] = df4['FriHoursClose'].apply(fixTime)
df4['SatHoursOpen'] = df4['SatHoursOpen'].apply(fixTime)
df4['SatHoursClose'] = df4['SatHoursClose'].apply(fixTime)
df4['SunHoursOpen'] = df4['SunHoursOpen'].apply(fixTime)
df4['SunHoursClose'] = df4['SunHoursClose'].apply(fixTime)

In [1]:
df4['MonHoursOpen'] = df4['MonHoursOpen'].asType(float)

NameError: ignored

In [0]:
df4.loc[:, 'MonHoursOpen':'SunHoursClose'] =df4.loc[:, 'MonHoursOpen':'SunHoursClose'].as

In [0]:
df4.head(2)

,business_id,latitude,longitude,stars,review_count,is_open,attributes.BikeParking,attributes.GoodForKids,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,fbcheckins,rating_count,engagement.count,pos_reviews,ambience.romantic,ambience.intimate,ambience.classy,ambience.hipster,ambience.divey,ambience.touristy,ambience.trendy,ambience.upscale,ambience.casual,cat_Acai Bowls,cat_Accessories,cat_Accountants,cat_Acne Treatment,cat_Active Life,cat_Adult Entertainment,cat_Afghan,cat_African,cat_Airport Lounges,cat_Alternative Medicine,cat_American (New),cat_American (Traditional),cat_Amusement Parks,cat_Animal Shelters,cat_Antiques,cat_Appliances,cat_Arabian,cat_Arcades,cat_Argentine,cat_Art Classes,cat_Art Galleries,cat_Art Schools,cat_Arts & Crafts,cat_Arts & Entertainment,cat_Asian Fusion,cat_Australian,cat_Austrian,cat_Automotive,cat_Bagels,cat_Bakeries,cat_Bangladeshi,cat_Bankruptcy Law,cat_Barbeque,cat_Bars,cat_Bartenders,cat_Basque,cat_Beaches,cat_Beauty & Spas,cat_Bed & Breakfast,cat_Beer,cat_Beer Bar,cat_Beer Hall,cat_Belgian,cat_Bistros,cat_Bocce Ball,cat_Books,cat_Bookstores,cat_Bowling,cat_Brasseries,cat_Brazilian,cat_Breakfast & Brunch,cat_Breweries,cat_Brewpubs,cat_British,cat_Bubble Tea,cat_Buffets,cat_Burgers,cat_Burmese,cat_Butcher,cat_Cabaret,cat_Cafes,cat_Cafeteria,cat_Cajun/Creole,cat_Cambodian,cat_Canadian (New),cat_Candy Stores,cat_Cannabis Clinics,cat_Cantonese,cat_Cards & Stationery,cat_Caribbean,cat_Casinos,cat_Caterers,cat_Champagne Bars,cat_Cheese Shops,cat_Cheesesteaks,cat_Chicken Shop,cat_Chicken Wings,cat_Chinese,cat_Chocolatiers & Shops,cat_Churches,cat_Cideries,cat_Cinema,cat_Cocktail Bars,cat_Coffee & Tea,cat_Coffee & Tea Supplies,cat_Coffee Roasteries,cat_Colombian,cat_Comedy Clubs,cat_Comfort Food,cat_Comic Books,cat_Community Centers,cat_Community Service/Non-Profit,cat_Convenience Stores,cat_Cooking Classes,cat_Cooking Schools,cat_Couriers & Delivery Services,cat_Creperies,cat_Cuban,cat_Custom Cakes,cat_Customized Merchandise,cat_Czech,cat_Czech/Slovakian,cat_Dance Clubs,cat_Day Spas,cat_Delicatessen,cat_Delis,cat_Department Stores,cat_Desserts,cat_Dim Sum,cat_Diners,cat_Dinner Theater,cat_Discount Store,cat_Dive Bars,cat_Do-It-Yourself Food,cat_Dog Walkers,cat_Donairs,cat_Donuts,cat_Dry Cleaning & Laundry,cat_Dumplings,cat_Education,cat_Egyptian,cat_Electricians,cat_Electronics,cat_Escape Games,cat_Ethical Grocery,cat_Ethiopian,cat_Ethnic Food,cat_Ethnic Grocery,cat_Event Planning & Services,cat_Eyelash Service,cat_Falafel,cat_Farmers Market,cat_Fashion,cat_Fast Food,cat_Festivals,cat_Filipino,cat_Financial Services,cat_Fish & Chips,cat_Fitness & Instruction,cat_Florists,cat_Flowers & Gifts,cat_Fondue,cat_Food,cat_Food Court,cat_Food Delivery Services,cat_Food Stands,cat_Food Tours,cat_Food Trucks,cat_French,cat_Fruits & Veggies,cat_Furniture Stores,cat_Gas Stations,cat_Gastropubs,cat_Gay Bars,cat_Gelato,cat_German,cat_Gift Shops,cat_Gluten-Free,cat_Greek,cat_Grocery,cat_Hainan,cat_Hair Loss Centers,cat_Hair Removal,cat_Hair Salons,cat_Haitian,cat_Hakka,cat_Halal,cat_Hawaiian,cat_Head Shops,cat_Health & Medical,cat_Health Markets,cat_Himalayan/Nepalese,cat_Hobby Shops,cat_Home & Garden,cat_Home Cleaning,cat_Home Decor,cat_Home Services,cat_Hong Kong Style Cafe,cat_Hookah Bars,cat_Hostels,cat_Hot Dogs,cat_Hot Pot,cat_Hotels,cat_Hotels & Travel,cat_Hungarian,cat_IT Services & Computer Repair,cat_Iberian,cat_Ice Cream & Frozen Yogurt,cat_Imported Food,cat_Indian,cat_Indonesian,cat_Indoor Playcentre,cat_International,cat_International Grocery,cat_Internet Cafes,cat_Investing,cat_Irish,cat_Irish Pub,cat_Italian,cat_Izakaya,cat_Japanese,cat_Jazz & Blues,cat_Jewelry,

In [0]:
df4.loc[:, 'MonHoursOpen':'SunHoursClose'] =df4.loc[:, 'MonHoursOpen':'SunHoursClose'].apply(fixTime)

AttributeError: ignored

In [0]:
df5 = df4

Drop old hours

In [0]:
df5 = df5.drop(['hours.Monday','hours.Tuesday','hours.Wednesday','hours.Thursday','hours.Friday','hours.Saturday','hours.Sunday'], axis=1)

In [0]:
df5 = df5.drop(['business_id'],axis=1)

In [0]:
df5.to_csv("Alldone_1hot_v0.csv")

In [12]:
pwd

'/content/drive/Shared drives/Data Science Principles Project'

In [24]:
#load Csv
%cd old\ data

/content/drive/Shared drives/Data Science Principles Project/old data


In [25]:
df5 = pd.read_csv("Alldone_1hot_v0.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,30,411,412,413,414,415) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
df5.head(5)

,Unnamed: 0,latitude,longitude,stars,review_count,is_open,attributes.BikeParking,attributes.GoodForKids,attributes.RestaurantsPriceRange2,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,fbcheckins,rating_count,engagement.count,pos_reviews,ambience.romantic,ambience.intimate,ambience.classy,ambience.hipster,ambience.divey,ambience.touristy,ambience.trendy,ambience.upscale,ambience.casual,cat_Acai Bowls,cat_Accessories,cat_Accountants,cat_Acne Treatment,cat_Active Life,cat_Adult Entertainment,cat_Afghan,cat_African,cat_Airport Lounges,cat_Alternative Medicine,cat_American (New),cat_American (Traditional),cat_Amusement Parks,cat_Animal Shelters,cat_Antiques,cat_Appliances,cat_Arabian,cat_Arcades,cat_Argentine,cat_Art Classes,cat_Art Galleries,cat_Art Schools,cat_Arts & Crafts,cat_Arts & Entertainment,cat_Asian Fusion,cat_Australian,cat_Austrian,cat_Automotive,cat_Bagels,cat_Bakeries,cat_Bangladeshi,cat_Bankruptcy Law,cat_Barbeque,cat_Bars,cat_Bartenders,cat_Basque,cat_Beaches,cat_Beauty & Spas,cat_Bed & Breakfast,cat_Beer,cat_Beer Bar,cat_Beer Hall,cat_Belgian,cat_Bistros,cat_Bocce Ball,cat_Books,cat_Bookstores,cat_Bowling,cat_Brasseries,cat_Brazilian,cat_Breakfast & Brunch,cat_Breweries,cat_Brewpubs,cat_British,cat_Bubble Tea,cat_Buffets,cat_Burgers,cat_Burmese,cat_Butcher,cat_Cabaret,cat_Cafes,cat_Cafeteria,cat_Cajun/Creole,cat_Cambodian,cat_Canadian (New),cat_Candy Stores,cat_Cannabis Clinics,cat_Cantonese,cat_Cards & Stationery,cat_Caribbean,cat_Casinos,cat_Caterers,cat_Champagne Bars,cat_Cheese Shops,cat_Cheesesteaks,cat_Chicken Shop,cat_Chicken Wings,cat_Chinese,cat_Chocolatiers & Shops,cat_Churches,cat_Cideries,cat_Cinema,cat_Cocktail Bars,cat_Coffee & Tea,cat_Coffee & Tea Supplies,cat_Coffee Roasteries,cat_Colombian,cat_Comedy Clubs,cat_Comfort Food,cat_Comic Books,cat_Community Centers,cat_Community Service/Non-Profit,cat_Convenience Stores,cat_Cooking Classes,cat_Cooking Schools,cat_Couriers & Delivery Services,cat_Creperies,cat_Cuban,cat_Custom Cakes,cat_Customized Merchandise,cat_Czech,cat_Czech/Slovakian,cat_Dance Clubs,cat_Day Spas,cat_Delicatessen,cat_Delis,cat_Department Stores,cat_Desserts,cat_Dim Sum,cat_Diners,cat_Dinner Theater,cat_Discount Store,cat_Dive Bars,cat_Do-It-Yourself Food,cat_Dog Walkers,cat_Donairs,cat_Donuts,cat_Dry Cleaning & Laundry,cat_Dumplings,cat_Education,cat_Egyptian,cat_Electricians,cat_Electronics,cat_Escape Games,cat_Ethical Grocery,cat_Ethiopian,cat_Ethnic Food,cat_Ethnic Grocery,cat_Event Planning & Services,cat_Eyelash Service,cat_Falafel,cat_Farmers Market,cat_Fashion,cat_Fast Food,cat_Festivals,cat_Filipino,cat_Financial Services,cat_Fish & Chips,cat_Fitness & Instruction,cat_Florists,cat_Flowers & Gifts,cat_Fondue,cat_Food,cat_Food Court,cat_Food Delivery Services,cat_Food Stands,cat_Food Tours,cat_Food Trucks,cat_French,cat_Fruits & Veggies,cat_Furniture Stores,cat_Gas Stations,cat_Gastropubs,cat_Gay Bars,cat_Gelato,cat_German,cat_Gift Shops,cat_Gluten-Free,cat_Greek,cat_Grocery,cat_Hainan,cat_Hair Loss Centers,cat_Hair Removal,cat_Hair Salons,cat_Haitian,cat_Hakka,cat_Halal,cat_Hawaiian,cat_Head Shops,cat_Health & Medical,cat_Health Markets,cat_Himalayan/Nepalese,cat_Hobby Shops,cat_Home & Garden,cat_Home Cleaning,cat_Home Decor,cat_Home Services,cat_Hong Kong Style Cafe,cat_Hookah Bars,cat_Hostels,cat_Hot Dogs,cat_Hot Pot,cat_Hotels,cat_Hotels & Travel,cat_Hungarian,cat_IT Services & Computer Repair,cat_Iberian,cat_Ice Cream & Frozen Yogurt,cat_Imported Food,cat_Indian,cat_Indonesian,cat_Indoor Playcentre,cat_International,cat_International Grocery,cat_Internet Cafes,cat_Investing,cat_Irish,cat_Irish Pub,cat_Italian,cat_Izakaya,cat_Japanese,cat_Jazz & Blues,cat_Jewelry,cat_Juice Bars & Smoothies,cat_Karaoke,cat_Kebab,cat_Kombucha,cat_Korean,cat_Kosher,cat_Landmarks & 

In [0]:
df5['MonHoursOpen'] = df5['MonHoursOpen'].astype(float)
df5['MonHoursClose'] = df5['MonHoursClose'].astype(float)
df5['TuesHoursOpen'] = df5['TuesHoursOpen'].astype(float)
df5['TuesHoursClose'] = df5['TuesHoursClose'].astype(float)
df5['WedHoursOpen'] = df5['WedHoursOpen'].astype(float)
df5['WedHoursClose'] = df5['WedHoursClose'].astype(float)
df5['ThursHoursOpen'] = df5['ThursHoursOpen'].astype(float)
df5['ThursHoursClose'] = df5['ThursHoursClose'].astype(float)
df5['FriHoursOpen'] = df5['FriHoursOpen'].astype(float)
df5['FriHoursClose'] = df5['FriHoursClose'].astype(float)
df5['SatHoursOpen'] = df5['SatHoursOpen'].astype(float)
df5['SatHoursClose'] = df5['SatHoursClose'].astype(float)
df5['SunHoursOpen'] = df5['SunHoursOpen'].astype(float)
df5['SunHoursClose'] = df5['SunHoursClose'].astype(float)

In [0]:
df6 = pd.get_dummies(df5)

In [34]:
pd.options.display.max_columns = None
display(df6)

,Unnamed: 0,latitude,longitude,stars,review_count,is_open,fbcheckins,rating_count,engagement.count,pos_reviews,ambience.romantic,ambience.intimate,ambience.classy,ambience.touristy,ambience.trendy,ambience.casual,cat_Acai Bowls,cat_Accessories,cat_Accountants,cat_Acne Treatment,cat_Active Life,cat_Adult Entertainment,cat_Afghan,cat_African,cat_Airport Lounges,cat_Alternative Medicine,cat_American (New),cat_American (Traditional),cat_Amusement Parks,cat_Animal Shelters,cat_Antiques,cat_Appliances,cat_Arabian,cat_Arcades,cat_Argentine,cat_Art Classes,cat_Art Galleries,cat_Art Schools,cat_Arts & Crafts,cat_Arts & Entertainment,cat_Asian Fusion,cat_Australian,cat_Austrian,cat_Automotive,cat_Bagels,cat_Bakeries,cat_Bangladeshi,cat_Bankruptcy Law,cat_Barbeque,cat_Bars,cat_Bartenders,cat_Basque,cat_Beaches,cat_Beauty & Spas,cat_Bed & Breakfast,cat_Beer,cat_Beer Bar,cat_Beer Hall,cat_Belgian,cat_Bistros,cat_Bocce Ball,cat_Books,cat_Bookstores,cat_Bowling,cat_Brasseries,cat_Brazilian,cat_Breakfast & Brunch,cat_Breweries,cat_Brewpubs,cat_British,cat_Bubble Tea,cat_Buffets,cat_Burgers,cat_Burmese,cat_Butcher,cat_Cabaret,cat_Cafes,cat_Cafeteria,cat_Cajun/Creole,cat_Cambodian,cat_Canadian (New),cat_Candy Stores,cat_Cannabis Clinics,cat_Cantonese,cat_Cards & Stationery,cat_Caribbean,cat_Casinos,cat_Caterers,cat_Champagne Bars,cat_Cheese Shops,cat_Cheesesteaks,cat_Chicken Shop,cat_Chicken Wings,cat_Chinese,cat_Chocolatiers & Shops,cat_Churches,cat_Cideries,cat_Cinema,cat_Cocktail Bars,cat_Coffee & Tea,cat_Coffee & Tea Supplies,cat_Coffee Roasteries,cat_Colombian,cat_Comedy Clubs,cat_Comfort Food,cat_Comic Books,cat_Community Centers,cat_Community Service/Non-Profit,cat_Convenience Stores,cat_Cooking Classes,cat_Cooking Schools,cat_Couriers & Delivery Services,cat_Creperies,cat_Cuban,cat_Custom Cakes,cat_Customized Merchandise,cat_Czech,cat_Czech/Slovakian,cat_Dance Clubs,cat_Day Spas,cat_Delicatessen,cat_Delis,cat_Department Stores,cat_Desserts,cat_Dim Sum,cat_Diners,cat_Dinner Theater,cat_Discount Store,cat_Dive Bars,cat_Do-It-Yourself Food,cat_Dog Walkers,cat_Donairs,cat_Donuts,cat_Dry Cleaning & Laundry,cat_Dumplings,cat_Education,cat_Egyptian,cat_Electricians,cat_Electronics,cat_Escape Games,cat_Ethical Grocery,cat_Ethiopian,cat_Ethnic Food,cat_Ethnic Grocery,cat_Event Planning & Services,cat_Eyelash Service,cat_Falafel,cat_Farmers Market,cat_Fashion,cat_Fast Food,cat_Festivals,cat_Filipino,cat_Financial Services,cat_Fish & Chips,cat_Fitness & Instruction,cat_Florists,cat_Flowers & Gifts,cat_Fondue,cat_Food,cat_Food Court,cat_Food Delivery Services,cat_Food Stands,cat_Food Tours,cat_Food Trucks,cat_French,cat_Fruits & Veggies,cat_Furniture Stores,cat_Gas Stations,cat_Gastropubs,cat_Gay Bars,cat_Gelato,cat_German,cat_Gift Shops,cat_Gluten-Free,cat_Greek,cat_Grocery,cat_Hainan,cat_Hair Loss Centers,cat_Hair Removal,cat_Hair Salons,cat_Haitian,cat_Hakka,cat_Halal,cat_Hawaiian,cat_Head Shops,cat_Health & Medical,cat_Health Markets,cat_Himalayan/Nepalese,cat_Hobby Shops,cat_Home & Garden,cat_Home Cleaning,cat_Home Decor,cat_Home Services,cat_Hong Kong Style Cafe,cat_Hookah Bars,cat_Hostels,cat_Hot Dogs,cat_Hot Pot,cat_Hotels,cat_Hotels & Travel,cat_Hungarian,cat_IT Services & Computer Repair,cat_Iberian,cat_Ice Cream & Frozen Yogurt,cat_Imported Food,cat_Indian,cat_Indonesian,cat_Indoor Playcentre,cat_International,cat_International Grocery,cat_Internet Cafes,cat_Investing,cat_Irish,cat_Irish Pub,cat_Italian,cat_Izakaya,cat_Japanese,cat_Jazz & Blues,cat_Jewelry,cat_Juice Bars & Smoothies,cat_Karaoke,cat_Kebab,cat_Kombucha,cat_Korean,cat_Kosher,cat_Landmarks & Historical Buildings,cat_Laotian,cat_Laser Hair Removal,cat_Latin American,cat_Laundromat,cat_Laundry Services,cat_Lawyers,cat_Lebanese,cat_Life Coach,cat_Live/Raw Food,cat_Local Flavor,cat_Local Services,cat_Lounges,cat_Macarons,cat_Mags,cat_Malaysian,cat_Marketing,cat_Massage,cat_Meat Shops,cat_Medical Spas,cat_Meditation Centers,cat_Mediterranean,cat_Men's Clothing,cat_Mexican,cat_Mid

In [0]:
df6.to_csv("Alldone_1hot_v1.csv")